In [1]:
import pandas as pd
from modules import *
from pluma.schema.vr import build_schema
%matplotlib widget

### 1. Load dataset

In [ ]:
datapicker = create_datapicker(show_summary=True, ubx=False, unity=True, create_geodata=True, schema=build_schema)
display(datapicker)

### 2. Explore Dataset Maps

In [ ]:
exploremap(datapicker.geodata)

### 3. Explore Dataset Streams

In [ ]:
plot_traces({
    'accelX': datapicker.dataset.streams.Accelerometer.data["Accl_X"],
    'angleX': datapicker.dataset.streams.Accelerometer.data["Orientation_X"],
    'altitude': datapicker.dataset.georeference.elevation,
})

### 4. Export Dataset to OGC API

In [6]:
record = DatasetRecord(datapicker.dataset, datapicker.geodata, properties=RecordProperties(
    title='<City> VR Walk: <CityRegion> Subject <ID>',
    description='VR walk data collection',
    license='CC BY-NC 4.0',
    tool='Bonsai',
    keywords=['<City>', 'VR', 'Walk', 'Biosignals'],
    contacts=[
        Contact(
            name='Your Name',
            institution='Your Institution',
            email='youremail@yourdomain.com'
        )
    ],
    themes=[]
))

In [7]:
rpath = Path(record.id)
export_geoframe_to_geojson(datapicker.geodata, rpath.with_suffix('.geojson'))
with open(rpath.with_suffix('.json'), 'w') as f:
    f.write(record.to_json())